In [1]:
from flask import Flask, request, make_response, jsonify
from flask_cors import CORS, cross_origin
from scipy.sparse import hstack
import pickle
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import joblib
from elasticsearch import Elasticsearch, helpers
import time
import pandas as pd
from scipy import sparse
from flask_jwt_extended import (
    create_access_token,
    get_jwt,
    jwt_required,
    JWTManager,
    set_access_cookies,
)
import lightgbm as lgbm
import numpy as np
import os


nltk.download("stopwords")
nltk.download("punkt")


[nltk_data] Downloading package stopwords to /home/amogus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/amogus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
app = Flask(__name__)
app.es_client = Elasticsearch(
    "https://127.0.0.1:9200",
    basic_auth=(
        "elastic",
        "yHcm1Pyq=jnDL_4gw93i",
    ),
    ca_certs="~/http_ca.crt",
)
app.recipes_df = pd.read_parquet("~/resources/food/recipes.parquet")
app.user_df = pd.read_parquet("~/resources/food/user.parquet")
app.folder_df = pd.read_parquet("~/resources/food/folder.parquet")
app.bookmark_df = pd.read_parquet("~/resources/food/bookmark.parquet")
app.keywords_df = pd.DataFrame.sparse.from_spmatrix(
    sparse.load_npz("/home/amogus/resources/food/keywords_df.npz"),
    columns=joblib.load("/home/amogus/resources/food/keywords_df_columns.joblib"),
)
app.recipes_numerical = pd.merge(
    app.recipes_df[["RecipeId", "AggregatedRating", "ReviewCount"]].fillna(0.0),
    app.keywords_df,
    left_index=True,
    right_index=True,
)

app.config["JWT_SECRET_KEY"] = "recipeme79"
app.config["JWT_TOKEN_LOCATION"] = ["cookies"]
app.config["JWT_COOKIE_SAMESITE"] = "None"
app.config["JWT_COOKIE_SECURE"] = True
app.config["JWT_COOKIE_CSRF_PROTECT"] = False
app.config["CORS_HEADERS"] = "Content-Type"
cors = CORS(app, resources={r"/foo": {"origins": "http://localhost:5173"}})
jwt = JWTManager(app)


recommend_query = {
    "function_score": {
        "query": {"match_all": {}},
        "functions": [
            {
                "script_score": {
                    "script": {
                        "source": "(doc['AggregatedRating'].value * doc['ReviewCount'].value + 4.813715683898263 * 5.316319710056667) / (doc['AggregatedRating'].value + 5.316319710056667)"
                    },
                },
                "weight": 1,
            },
            {"random_score": {}, "weight": 20},
        ],
        "score_mode": "avg",
    }
}


def get_search_query(query: str):
    return {
        "function_score": {
            "query": {
                "dis_max": {
                    "queries": [
                        {"match": {"Name": query}},
                        {"match": {"RecipeIngredientParts": query}},
                        {"match": {"RecipeInstructions": query}},
                        {"match": {"Keywords": query}},
                    ],
                    "tie_breaker": 0.3,
                }
            },
            "functions": [
                {
                    "script_score": {
                        "script": {
                            "source": "(doc['AggregatedRating'].value * doc['ReviewCount'].value + 4.813715683898263 * 5.316319710056667) / (doc['AggregatedRating'].value + 5.316319710056667)"
                        },
                    },
                    "weight": 1,
                },
                {
                    "script_score": {
                        "script": {"source": "_score"},
                    },
                    "weight": 1,
                },
            ],
            "score_mode": "multiply",
        }
    }


def train_model():
    feature_columns = app.recipes_numerical.columns.delete(0)
    folders_recipes_df = pd.merge(
        app.bookmark_df.sort_values(["folder_index", "rating"]),
        app.recipes_numerical,
        how="inner",
        left_on="RecipeId",
        right_on="RecipeId",
    )
    target_column = ["rating"]
    groups = folders_recipes_df["folder_index"].value_counts().sort_index().to_list()
    app.ranker.fit(
        folders_recipes_df[feature_columns],
        folders_recipes_df[target_column],
        group=groups,
    )
    joblib.dump(app.ranker, "/home/amogus/resources/food/ranker.joblib")


if os.path.isfile("/home/amogus/resources/food/ranker.joblib"):
    app.ranker = joblib.load("/home/amogus/resources/food/ranker.joblib")
else:
    app.ranker = lgbm.LGBMRanker(
        min_child_samples=1,
        num_leaves=63,
        learning_rate=0.01,
        n_estimators=1000,
        verbosity=-1,
    )
    train_model()


def recommend_from_folder_index(index: int):
    folder_keywords = np.unique(
        pd.merge(
            app.bookmark_df[app.bookmark_df["folder_index"] == index],
            app.recipes_df,
            how="inner",
            left_on="RecipeId",
            right_on="RecipeId",
        )["Keywords"].explode()
    )
    exclude_kw = list(set(app.keywords_df.columns).difference(folder_keywords))
    pred_df = app.recipes_numerical.copy()
    pred_df = pred_df.loc[pred_df[exclude_kw].values.sum(axis=1) == 0]
    preds = app.ranker.predict(pred_df[app.recipes_numerical.columns.delete(0)])
    return app.recipes_df.iloc[
        pd.concat(
            [pred_df.reset_index(), pd.Series(preds).to_frame("rank_score")], axis=1
        )
        .sort_values("rank_score", ascending=False)
        .head(9)
        .sample(frac=1)
        .head(6)
        .set_index("index")
        .index
    ]


def recommend_from_user(user: str):
    folder_keywords = np.unique(
        pd.merge(
            app.bookmark_df[
                app.bookmark_df["folder_index"].isin(
                    app.folder_df[app.folder_df["username"] == user].index
                )
            ],
            app.recipes_df,
            how="inner",
            left_on="RecipeId",
            right_on="RecipeId",
        )["Keywords"].explode()
    )
    exclude_kw = list(set(app.keywords_df.columns).difference(folder_keywords))
    pred_df = app.recipes_numerical.copy()
    pred_df = pred_df.loc[pred_df[exclude_kw].values.sum(axis=1) == 0]
    preds = app.ranker.predict(pred_df[app.recipes_numerical.columns.delete(0)])
    return app.recipes_df.iloc[
        pd.concat(
            [pred_df.reset_index(), pd.Series(preds).to_frame("rank_score")], axis=1
        )
        .sort_values("rank_score", ascending=False)
        .head(9)
        .sample(frac=1)
        .head(6)
        .set_index("index")
        .index
    ]


def gen_recommendations(user: str):
    recommend_result = dict()
    if not app.folder_df[app.folder_df["username"] == user].empty:
        if pd.merge(
            app.folder_df[app.folder_df["username"] == user],
            app.bookmark_df,
            left_index=True,
            right_on="folder_index",
            how="inner",
        ).empty:
            recommend_result["recommend_from_summary"] = {"results": []}
        else:
            recommend_result["recommend_from_summary"] = {
                "results": recommend_from_user(user)
                .fillna(np.nan)
                .replace([np.nan], [None])
                .to_dict(orient="records")
            }
            for i, outt in enumerate(
                recommend_result["recommend_from_summary"]["results"]
            ):
                for k, v in outt.items():
                    if isinstance(v, np.ndarray):
                        recommend_result["recommend_from_summary"]["results"][i][k] = (
                            list(v)
                        )
        folder = app.folder_df[app.folder_df["username"] == user].sample(frac=1).head(1)
        if pd.merge(
            folder,
            app.bookmark_df,
            left_index=True,
            right_on="folder_index",
            how="inner",
        ).empty:
            recommend_result["recommend_from_folder"] = {
                "folder_name": folder.iloc[0]["folder_name"],
                "results": [],
            }
        else:
            recommend_result["recommend_from_folder"] = {
                "folder_name": folder.iloc[0]["folder_name"],
                "results": recommend_from_folder_index(folder.index[0])
                .fillna(np.nan)
                .replace([np.nan], [None])
                .to_dict(orient="records"),
            }
            for i, outt in enumerate(
                recommend_result["recommend_from_folder"]["results"]
            ):
                for k, v in outt.items():
                    if isinstance(v, np.ndarray):
                        recommend_result["recommend_from_folder"]["results"][i][k] = (
                            list(v)
                        )
        recommend_result["recommend_from_random"] = {
            "results": app.recipes_df.sample(6)
            .fillna(np.nan)
            .replace([np.nan], [None])
            .to_dict(orient="records")
        }
        for i, outt in enumerate(recommend_result["recommend_from_random"]["results"]):
            for k, v in outt.items():
                if isinstance(v, np.ndarray):
                    recommend_result["recommend_from_random"]["results"][i][k] = list(v)
    return recommend_result


def create_folder(username: string, folder_name: string):
    app.folder_df = pd.concat(
        [
            app.folder_df,
            pd.DataFrame(
                [[username, folder_name]], columns=["username", "folder_name"]
            ),
        ],
        ignore_index=True,
    )
    app.folder_df.to_parquet("~/resources/food/folder.parquet")
    train_model()
    return app.folder_df


def get_folders_username(username: string):
    return app.folder_df[app.folder_df["username"] == username].reset_index()


def get_recipes_folder(id: int):
    return pd.merge(
        app.bookmark_df[app.bookmark_df["folder_index"] == id],
        app.recipes_df,
        how="inner",
        left_on="RecipeId",
        right_on="RecipeId",
    )[["rating"] + list(app.recipes_df.columns)]


def bookmark_recipe(folder_index: int, RecipeID: int, rating: float):
    if not app.bookmark_df[
        (app.bookmark_df["folder_index"] == folder_index)
        & (app.bookmark_df["RecipeId"] == RecipeID)
    ].empty:
        app.bookmark_df.loc[
            (app.bookmark_df["folder_index"] == folder_index)
            & (app.bookmark_df["RecipeId"] == RecipeID),
            "rating",
        ] = rating
    else:
        app.bookmark_df = pd.concat(
            [
                app.bookmark_df,
                pd.DataFrame(
                    [[folder_index, RecipeID, rating]],
                    columns=["folder_index", "RecipeId", "rating"],
                ),
            ],
            ignore_index=True,
        )
    app.bookmark_df.to_parquet("~/resources/food/bookmark.parquet")
    train_model()
    return app.bookmark_df


@app.route("/folder", methods=["POST"])
@cross_origin(origin="localhost", headers=["Content-Type"])
@jwt_required()
def create_folder_request():
    claims = get_jwt()
    folder_name = request.json.get("folder_name", None)
    start = time.time()
    response_object = {"status": "success"}
    print(claims.get("sub"), folder_name)
    create_folder(claims.get("sub"), folder_name)
    end = time.time()
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/bookmark", methods=["POST"])
@cross_origin(origin="localhost", headers=["Content-Type"])
@jwt_required()
def add_bookmark_to_folder():
    folder_index = request.json.get("folder_index", None)
    recipe_id = request.json.get("recipe_id", None)
    rating = request.json.get("rating", None)
    start = time.time()
    response_object = {"status": "success"}
    if folder_index is not None and recipe_id is not None and rating is not None:
        bookmark_recipe(folder_index, recipe_id, rating)
    else:
        response_object = {"status": "fail"}
    end = time.time()
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/get_recommendations", methods=["GET"])
@jwt_required()
def recommend_user():
    claims = get_jwt()
    start = time.time()
    response_object = {"status": "success"}
    results = gen_recommendations(claims.get("sub"))
    end = time.time()
    response_object["results"] = results
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/folder_recommend", methods=["GET"])
@jwt_required()
def recommend_one_folder():
    start = time.time()
    response_object = {"status": "success"}
    argList = request.args.to_dict(flat=False)
    folder_index = argList["folder_index"][0]
    results = recommend_from_folder_index(int(folder_index))
    end = time.time()
    response_object["total_hit"] = len(results)
    response_object["results"] = (
        results.fillna(np.nan).replace([np.nan], [None]).to_dict("records")
    )
    for i, outt in enumerate(response_object["results"]):
        for k, v in outt.items():
            if isinstance(v, np.ndarray):
                response_object["results"][i][k] = list(v)
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/folder", methods=["GET"])
@jwt_required()
def view_one_folder():
    start = time.time()
    response_object = {"status": "success"}
    argList = request.args.to_dict(flat=False)
    id = argList["id"][0]
    results = get_recipes_folder(int(id))
    end = time.time()
    response_object["total_hit"] = len(results)
    response_object["results"] = (
        results.fillna(np.nan).replace([np.nan], [None]).to_dict("records")
    )
    for i, outt in enumerate(response_object["results"]):
        for k, v in outt.items():
            if isinstance(v, np.ndarray):
                response_object["results"][i][k] = list(v)
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/folders", methods=["GET"])
@jwt_required()
def view_all_folders():
    start = time.time()
    claims = get_jwt()
    response_object = {"status": "success"}
    folders = get_folders_username(claims.get("sub")).to_dict(orient="records")
    for folder in folders:
        recipes = (
            get_recipes_folder(folder["index"])
            .fillna(np.nan)
            .replace([np.nan], [None])
            .sort_values("rating", ascending=False)
            .to_dict(orient="records")
        )
        for i, outt in enumerate(recipes):
            for k, v in outt.items():
                if isinstance(v, np.ndarray):
                    recipes[i][k] = list(v)
        folder["recipes"] = recipes
    end = time.time()
    response_object["total_hit"] = len(folders)
    response_object["results"] = folders
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/user-detail", methods=["GET"])
@jwt_required()
def get_jwt_data():
    claims = get_jwt()
    return make_response(jsonify(dict(claims)), 200)


@app.route("/login", methods=["POST"])
@cross_origin(origin="localhost", headers=["Content-Type"])
def login():
    username = request.json.get("username", None)
    password = request.json.get("password", None)

    try:
        user = (
            app.user_df.reset_index()[
                (app.user_df["username"] == username)
                & (app.user_df["password"] == password)
            ]
            .iloc[0]
            .to_dict()
        )
        print(user)
        additional_claims = {"disp": user.get("display_name")}
        access_token = create_access_token(
            identity=user.get("username"), additional_claims=additional_claims
        )
        resp = make_response(jsonify({}), 200)
        set_access_cookies(resp, access_token)
        return resp
    except IndexError:
        return make_response(jsonify({"msg": "Bad username or password"}), 401)


@app.route("/register", methods=["POST"])
@cross_origin(origin="localhost", headers=["Content-Type"])
def register():
    username = request.json.get("username", None)
    password = request.json.get("password", None)
    display_name = request.json.get("display_name", None)

    if app.user_df[(app.user_df["username"] == "username")].shape[0] > 0:
        return make_response(jsonify({"msg": "Bad username"}), 401)
    else:
        app.user_df = pd.concat(
            [
                app.user_df,
                pd.DataFrame(
                    [[username, password, display_name]],
                    columns=["username", "password", "display_name"],
                ),
            ],
            ignore_index=True,
        )
        app.user_df.to_parquet("~/resources/food/user.parquet")
        additional_claims = {"disp": display_name}
        access_token = create_access_token(
            identity=username, additional_claims=additional_claims
        )
        resp = make_response(jsonify({}), 200)
        set_access_cookies(resp, access_token)
        return resp


@app.route("/recommended", methods=["GET"])
@jwt_required()
def get_recommended():
    start = time.time()
    response_object = {"status": "success"}
    results = app.es_client.search(
        index="recipes",
        size=6,
        query=recommend_query,
    )
    end = time.time()
    total_hit = results["hits"]["total"]["value"]
    results_df = pd.DataFrame(
        [[hit["_score"], *hit["_source"].values()] for hit in results["hits"]["hits"]],
        columns=["score"] + list(results["hits"]["hits"][0]["_source"].keys()),
    )
    response_object["total_hit"] = total_hit
    response_object["results"] = results_df.to_dict("records")
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/search", methods=["GET"])
@jwt_required()
def search():
    start = time.time()
    response_object = {"status": "success"}
    argList = request.args.to_dict(flat=False)
    query = argList["query"][0]
    results = app.es_client.search(
        index="recipes",
        size=12,
        query=get_search_query(query),
    )
    end = time.time()
    total_hit = results["hits"]["total"]["value"]
    if len(results["hits"]["hits"]) > 0:
        results_df = pd.DataFrame(
            [
                [hit["_score"], *hit["_source"].values()]
                for hit in results["hits"]["hits"]
            ],
            columns=["score"] + list(results["hits"]["hits"][0]["_source"].keys()),
        )
    else:
        results_df = pd.DataFrame()
    response_object["total_hit"] = total_hit
    response_object["results"] = results_df.to_dict("records")
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/recipes/<int:id>", methods=["GET"])
@jwt_required()
def get_by_id(id: int):
    start = time.time()
    response_object = {"status": "success"}
    result = app.es_client.get(index="recipes", id=id)
    end = time.time()
    result_df = pd.DataFrame(
        [[*result["_source"].values()]],
        columns=list(result["_source"].keys()),
    )
    response_object["results"] = result_df.to_dict("records")
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/suggest", methods=["GET"])
@jwt_required()
def suggest():
    start = time.time()
    response_object = {"status": "success"}
    argList = request.args.to_dict(flat=False)
    query = argList["query"][0]
    query_dictionary = {
        "suggest": {
            "text": query,
            "suggest-1": {"term": {"field": "all_texts"}},
            "suggest-2": {"term": {"field": "Name"}},
            "suggest-3": {"term": {"field": "Description"}},
            "suggest-4": {"term": {"field": "RecipeInstructions"}},
        }
    }
    res = app.es_client.search(index="recipes", body=query_dictionary)

    p = []
    for term in np.array(list(res["suggest"].values())).T:
        result = {}
        result["text"] = term[0]["text"]
        options = [v["options"] for v in term]
        result["candidates"] = {}
        for option in options:
            candidates = {}
            if len(option) > 0:
                candidates["text"] = option[0]["text"]
                for candidate in option:
                    # print(candidate)
                    if candidate["text"] not in result["candidates"]:
                        result["candidates"][candidate["text"]] = {
                            "score": candidate["score"],
                            "freq": candidate["freq"],
                        }
                    else:
                        result["candidates"][candidate["text"]]["score"] = (
                            result["candidates"][candidate["text"]]["score"]
                            * result["candidates"][candidate["text"]]["freq"]
                            + candidate["score"] * candidate["freq"]
                        ) / (
                            result["candidates"][candidate["text"]]["freq"]
                            + candidate["freq"]
                        )
                        result["candidates"][candidate["text"]]["freq"] = (
                            result["candidates"][candidate["text"]]["freq"]
                            + candidate["freq"]
                        )
        p += [result["candidates"]]
    out = [""] * len(query.split())
    for i, pp in enumerate(p):
        if pp:
            df = pd.DataFrame.from_dict(pp, orient="index")
            R = (df["score"] * df["freq"]).sum() / df["freq"].sum()
            W = df["freq"].mean()
            df["bayes_score"] = (df["score"] * df["freq"] + W * R) / (df["freq"] + W)
            out[i] = df.sort_values("bayes_score", ascending=False).head(1).index[0]
        else:
            out[i] = query.split()[i]
    end = time.time()
    response_object["suggest"] = " ".join(out)
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.after_request
def apply_caching(response):
    response.headers["Access-Control-Allow-Origin"] = "http://localhost:5173"
    response.headers["Access-Control-Allow-Headers"] = (
        "Origin, X-Requested-With, Content-Type, Accept"
    )
    response.headers["Access-Control-Allow-Credentials"] = "true"L
    return response


In [3]:
app.run(debug=False, host="0.0.0.0", ssl_context="adhoc")


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:5000
 * Running on https://10.148.0.2:5000
Press CTRL+C to quit
183.89.186.111 - - [18/Mar/2024 09:25:03] "GET /recommended HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:25:03] "GET /folders HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:25:03] "GET /get_recommendations HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:25:03] "GET /user-detail HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:28:11] "GET /get_recommendations HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:28:20] "GET /recommended HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:28:20] "GET /folders HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:28:20] "GET /get_recommendations HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:28:20] "GET /user-detail HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:28:23] "GET /get_recommendations HTTP/1.1" 401 -
183.89.186.111 - - [18/Mar/2024 09:28:29] "GET /get_recommendations HTTP/1.1" 401 

{'index': 0, 'username': 'test', 'password': 'test', 'display_name': 'test-user'}


183.89.186.111 - - [18/Mar/2024 09:31:34] "GET /recommended HTTP/1.1" 200 -
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_

{'index': 0, 'username': 'test', 'password': 'test', 'display_name': 'test-user'}


...

183.89.186.111 - - [18/Mar/2024 10:05:37] "GET /recommended HTTP/1.1" 200 -
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_

{'index': 0, 'username': 'test', 'password': 'test', 'display_name': 'test-user'}


183.89.186.111 - - [18/Mar/2024 10:23:53] "GET /recommended HTTP/1.1" 200 -
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_

{'index': 0, 'username': 'test', 'password': 'test', 'display_name': 'test-user'}


/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(np.nan)
/tmp/ipykernel_8353/1410142007.py:399